<a href="https://colab.research.google.com/github/ANUJSELF/-_-1-/blob/main/Copy_of_Solving_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Business Problems with AI

## Objective
Develop a proof of concept application that intelligently processes email order requests and customer inquiries for a fashion store. The system should categorize emails into product inquiries or order requests and generate appropriate responses based on product catalog information and stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

## Instructions

- All requirements have to be implemented with the use of LLMs to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Because solving this assessment requires using advanced LLM capabilities, we provide you with a temporary OpenAI API key granting access to GPT-4o. You can use this key or your own, but please note that the provided key has a token quota, so use it wisely. We have carefully adjusted the limits to ensure they are sufficient for you to complete the task.
- Address the requirements in the order listed. Please review the requirements ahead to have a general implementation plan in place before you start.
- Your deliverables should include the code developed within this notebook, and a spreadsheet containing results across separate sheets. Comments detailing your thought process are appreciated.
- You are allowed to use additional libraries (langchain, etc.) in order to make solution more concise.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output:**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process order requests in the order they are received. For each request, verify product availability in stock. If the order can be fully fulfilled, create a new order line with the status **created**. If the order cannot be fully fulfilled, create a line with the status **out of stock**. After placing the order, update the stock to accurately reflect the current inventory levels.

    **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate and save response emails based on order processing results. Depending on the order status email should inform customer that their order was processed or could not be fulfilled. If the order was successfully processed, send an email to the customer informing them that their order has been processed, including details like the product name and quantity. If the order could not be fulfilled due to insufficient stock, send an email explaining the situation and specifying which items are out of stock. Optionally, offer options such as waiting for restock or choosing alternative products. Do your best to make an email look production ready!

    **Output**: Populate the **order-response** sheet with columns: email ID, response.

### 3. Handle product inquiry

Customers may ask general open questions.
- Respond to product inquiries using the information from the product catalog.
- Ensure that solution you provide scales well to handle the full catalog of over 100,000 products. This means that solutions which include raw product catalog in the prompt will be rated lower!

**Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria

- **Utilization of AI Tools**: Effectiveness in leveraging AI technologies provided (e.g., OpenAI API) to fulfill the task requirements. Employ your knowledge of various AI driven development techniques depending on the task.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational.
- **Code Quality and Clarity**: Code should be well-organized with clear logic and structure.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the spreadsheet. Double check before submitting!

We look forward to seeing your solution and how you approach solving real-world problems using AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [13]:
# Install the OpenAI Python package.
%pip install openai

In [14]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0BIj000001JD2CMAW'
)


def ai_chat(query):
  completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": query}
  ]
)
  return (completion.choices[0].message.content)

In [15]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Task 1. Classify emails

In [16]:
email_list = emails_df.set_index('email_id')['subject'].to_dict()

promt = "You are a data analyst for a ecom store I am giving you an object with key value pairs of email id and their subjects sort these emails into - product enquiry or order request by reading and understanding the subject of each mail and based on what its language and what it infers. For example if subject mentions a product's name it is likely a inquiry and if a mail subject mentions any verb that implies purchasing or placing order then it is an order request. Give me the output in the same format as input just add a value of type to each object which refer to its classification, refrain from outputting anything extra just give me the data.  Here is the list : "
q1 = promt + str(email_list)
data_res = (ai_chat(q1))
data_obj1 = data_res[9:-3]

In [17]:
import pandas as pd
import re

def string_to_dataframe(data_string):
    # Use regex to extract key-value pairs
    pattern = r"'(E\d+)':\s*{[^}]*'subject':\s*([^,}]+)[^}]*'type':\s*([^}]+)}"
    matches = re.findall(pattern, data_string)

    data = []
    for match in matches:
        id, subject, type = match
        subject = subject.strip("'")
        type = type.strip().strip("'")
        if type.lower() == 'none':
            type = None
        data.append({
            'id': id,
            'subject': subject if subject.lower() != 'nan' else None,
            'type': type
        })

    return pd.DataFrame(data)

email_classifications_df = string_to_dataframe(data_obj1)

# Task 2. Process order requests

In [18]:
def extract_product_info(email_body):
  """Extracts product ID and quantity from email body using GPT-4o.

  Args:
    email_body: The email body text.

  Returns:
    tuple: (product_id, quantity)
  """

  # Replace with your GPT-4o prompt and logic to extract product information
  prompt = f"This is an email requesting to order products. Based on the content, identify the product ID and quantity. \n Email body: {email_body}"
  completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
          {"role": "user", "content": prompt}
      ]
  )
  extracted_info = completion.choices[0].message.content
  # Parse the extracted information to get product ID and quantity (regular expressions or string manipulation)
  product_id, quantity = parse_extracted_info(extracted_info)
  return product_id, quantity

In [19]:
def process_orders(emails_df, products_df):
  """Processes order requests, updates stock, and generates responses.

  Args:
    emails_df: DataFrame containing email data with columns: email ID, category, body.
    products_df: DataFrame containing product data with columns: product ID, name, stock amount.

  Returns:
    order_status_df: DataFrame with order status information.
    order_responses_df: DataFrame with order response content.
  """

  order_status = []
  order_responses = []

  for index, email in emails_df.iterrows():
    if email['category'] == 'order request':
      # Extract product ID and quantity using your implemented function
      try:
        product_id, quantity = extract_product_info(email['body'])
      except Exception as e:
        print(f"Error extracting product information for email {email['email ID']}: {e}")
        continue

      # Find product in the catalog
      product = products_df[products_df['product ID'] == product_id]
      if product.empty:
        # Handle product not found case (e.g., inform customer)
        order_responses.append({'email ID': email['email ID'], 'response': f"Product with ID {product_id} is not found in our catalog."})
        continue

      product = product.iloc[0]

      if product['stock amount'] >= quantity:
        order_status.append({'email ID': email['email ID'], 'product ID': product_id, 'quantity': quantity, 'status': 'created'})
        products_df.loc[products_df['product ID'] == product_id, 'stock amount'] -= quantity
        order_responses.append({'email ID': email['email ID'], 'response': f"Your order for {quantity} units of {product['name']} has been processed."})
      else:
        order_status.append({'email ID': email['email ID'], 'product ID': product_id, 'quantity': quantity, 'status': 'out of stock'})
        order_responses.append({'email ID': email['email ID'], 'response': f"Unfortunately, your order for {product['name']} cannot be fulfilled due to insufficient stock."})

  return pd.DataFrame(order_status), pd.DataFrame(order_responses)

# Example usage with error handling
try:
  order_status_df, order_responses_df = process_orders(emails_df, products_df)
  print(order_status_df)
  print(order_responses_df)
except Exception as e:
  print(f"An error occurred during order processing: {e}")

An error occurred during order processing: 'category'


# Task 3. Handle product inquiry

In [20]:
import openai

def handle_inquiries(emails_df, products_df):
  """Handles product inquiries using an LLM and product catalog.

  Args:
    emails_df: DataFrame containing email data with columns: email ID, category, body.
    products_df: DataFrame containing product data with columns: product ID, name, description, etc.

  Returns:
    inquiry_responses_df: DataFrame with inquiry responses.
  """

  inquiry_responses = []

  for _, email in emails_df.iterrows():
    if email['category'] == 'product inquiry':
      email_body = email['body']

      # Create a prompt for the LLM
      prompt = f"""
      Provide a comprehensive and informative response to the following product inquiry, leveraging the provided product information:

      Inquiry: {email_body}

      Product Information:
      {products_df.to_json(orient='records')}
      """

      # Use your LLM to generate a response
      response = openai.ChatCompletion.create(
          model="gpt-4",  # Replace with your desired LLM
          messages=[
              {"role": "user", "content": prompt}
          ]
      )
      response_text = response.choices[0].message.content

      inquiry_responses.append({'email ID': email['email ID'], 'response': response_text})

  return pd.DataFrame(inquiry_responses)

###Seperate Spreadsheet.

In [21]:
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import openai

